In [1]:
import torch
from transformers import pipeline
import pandas as pd

# Load the embeddings from the pickle file
file_path = "..\\Data\\processed\\embeddings_10k.pkl"
sample_data = pd.read_pickle(file_path)

# Display the first few rows of the data to understand its structure
sample_data.head()

,user_id,conversations,company_name,cleaned_conversations,embeddings
0,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[-0.39655936, -0.23127136, 0.11331421, 0.30680..."
1,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[0.025823694, -0.13674489, -0.062481128, 0.173..."
2,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[-0.3794237, -0.15264028, -0.12576532, 0.23881..."
3,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare,Customer how can I get in touch with someone t...,"[-0.109025486, 0.015738899, -0.0615948, 0.0280..."
4,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare,Customer how can I get in touch with someone t...,"[-0.18445823, -0.03162857, -0.0017102931, 0.02..."


In [4]:
sample_data

,user_id,conversations,company_name,cleaned_conversations,embeddings
0,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[-0.39655936, -0.23127136, 0.11331421, 0.30680..."
1,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[0.025823694, -0.13674489, -0.062481128, 0.173..."
2,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[-0.3794237, -0.15264028, -0.12576532, 0.23881..."
3,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare,Customer how can I get in touch with someone t...,"[-0.109025486, 0.015738899, -0.0615948, 0.0280..."
4,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare,Customer how can I get in touch with someone t...,"[-0.18445823, -0.03162857, -0.0017102931, 0.02..."
...,...,...,...,...,...
33791,126216,"Customer: @GWRHelp no info, full platform - no...",GWRHelp,Customer no info full platform no idea when th...,"[0.10950682, -0.33966443, 0.039041705, 0.22977..."
33792,126216,Customer: @GWRHelp I'm on one of your trains t...,GWRHelp,Customer Im on one of your trains that is actu...,"[-0.07753325, -0.21089466, 0.01709967, 0.36077..."
33793,126216,Customer: I feel for you @GWRHelp social media...,GWRHelp,Customer I feel for you social media team ther...,"[-0.132401, -0.12074328, 0.11453965, 0.0051263..."
33794,126216,Customer: @GWRHelp another evening of disrupti...,GWRHelp,Customer another evening of disruptions from y...,"[-0.023492428, -0.09045017, 0.16090254, 0.2769..."


In [2]:
# Load the Llama pipeline
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Define a function for entity extraction
def extract_entities(tweet_text):
    messages = [
        {"role": "system", "content": "You are an assistant that extracts entities from text."},
        {"role": "user", "content": f"Identify all relevant entities in this tweet, including product, service, and issue type: '{tweet_text}'"},
    ]
    output = pipe(messages, max_new_tokens=256)
    return output[0]["generated_text"]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Define a function for entity extraction
def extract_entities(tweet_text):
    messages = [
        {"role": "system", "content": "You are an assistant that extracts entities from text."},
        {"role": "user", "content": f"Identify all relevant entities in this tweet, including product, service, and issue type: '{tweet_text}'"},
    ]
    output = pipe(messages, max_new_tokens=256)
    return output[0]["generated_text"]

# Get the first 3 tweets for testing
first_three_tweets = sample_data["cleaned_conversations"].head(3).tolist()

# Apply entity extraction to the first 3 tweets
extracted_entities = [extract_entities(tweet) for tweet in first_three_tweets]

# Print the results
for idx, (tweet, entities) in enumerate(zip(first_three_tweets, extracted_entities)):
    print(f"Tweet {idx + 1}:")
    print(f"Original: {tweet}")
    print(f"Entities: {entities}")
    print("\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Tweet 1:
Original: Customer is the worst customer service
Company I would love the chance to review the account and provide assistance
Entities: [{'role': 'system', 'content': 'You are an assistant that extracts entities from text.'}, {'role': 'user', 'content': "Identify all relevant entities in this tweet, including product, service, and issue type: 'Customer is the worst customer service\nCompany I would love the chance to review the account and provide assistance'"}, {'role': 'assistant', 'content': 'Here are the relevant entities extracted from the tweet:\n\n1. **Issue Type**: Poor customer service\n2. **Product/Service**: Account ( likely referring to a bank or financial account)\n\nNote that the term "customer service" is not explicitly mentioned in the tweet, but it can be inferred as the entity being referred to.'}]


Tweet 2:
Original: Customer is the worst customer service
Company Can you please send us a private message so that I can gain further details about your account
